In [ ]:
import io
import re
import json
from datetime import datetime
from collections import defaultdict, namedtuple
from pathlib import Path
from ghapi.all import gh2date

from IPython.display import Markdown, display
from tqdm.notebook import tqdm
from markdown import markdown

import hashlib
from os import environ
import pyperclip
import subprocess
import requests

import githelpers
import ghapicache
import release_notes

%load_ext autoreload
%autoreload 1
%aimport githelpers
%aimport ghapicache
%aimport release_notes

In [ ]:
cached = ghapicache.GhApiCache()
ghapi = cached.api

In [ ]:
# Load teams
TEAMS = {}
for t in tqdm(cached.teams()):
    TEAMS[t['name']] = release_notes.get_team(cached, t)

team_members = frozenset(chain(*(t.members for t in TEAMS.values())))
org_members = frozenset(m['login'] for m in cached.org_members())
TEAMS['affiliates'] = release_notes.Team(description="Associated with Celeritas but not core members",
                                         members=(org_members - team_members))

# Initialize the user cache
local_repo = Path("/Users/seth/Code/celeritas-temp")
user_cache = release_notes.UserCache(cached, local_repo / "scripts/release/users.json")
for login in tqdm(org_members):
    user_cache[login]

# List active members

This is to be used for crediting in presentations, etc.

In [ ]:
text = []
for team, title in [
    ('code-lead', 'Code lead'),
    ('core', 'Core members'),
    ('core-advisor', 'Core advisors'),
    ('affiliates', 'Affiliates'),]:
    text.append(f"## {title}")
    m = [user_cache[username] for username in TEAMS[team].members]
    for member in sorted(m, key=release_notes.get_last_name):
        text.append("- " + release_notes.format_user(member))
    text.append("")

display(Markdown("\n".join(text)))

## Release note generation

- Merge base should be all commits *already* released (skip documenting)
- Target branch is the one where the release candidate is
- Previous major branch allows all "v.x" contributors to be credited

In [ ]:
ReleaseMetadata = release_notes.ReleaseMetadata

# All commits from all time
all_md = ReleaseMetadata(
    merge_bases=['v0.0.0'],
)

all_prs = release_notes.PullRequestRange(all_md)
count_contrib = release_notes.ContributionCounter(cached)
for pr in tqdm(all_prs.pull_ids):
    count_contrib(pr)
authors = count_contrib.sorted().author
authors

In [ ]:
print("Missing ORCIDs:")
print(" ".join(f"@{u}" for u in authors if user_cache[u].orcid is None))

## Release

In [ ]:
# Major release
major_md = ReleaseMetadata(
    release='0.6.0',
    merge_bases=['v0.5.0', 'v0.5.2'],
)

# Minor release
minor_md = ReleaseMetadata(
    release='0.5.3',
    merge_bases=['v0.5.2'],
    target_branch='backports/v0.5'
)

In [ ]:
if 0:
    # Backport release:
    release_md = minor_md
    note_body =  """
Version {release} is a minor update to Celeritas featuring improved diagnostic
output, improved CMake integration for VecGeom+CUDA, and several bug fixes.
"""
else:
    # Major release
    release_md = major_md
    note_body = """
Version {release} is a major update to Celeritas featuring:

- 

A few minor features are noteworthy:

- 

Important changes:

- 

Notable bug fixes include:

- 

Some interfaces have been removed:

- 
"""

In [ ]:

prs = release_notes.PullRequestRange(release_md)
sorted_pulls = release_notes.SortedPulls(cached)
count_contrib = release_notes.ContributionCounter(cached)
for pr_id in tqdm(prs.pull_ids):
    try:
        count_contrib(pr_id)
        sorted_pulls.add(pr_id)
    except Exception as e:
        print(f"Error adding PR #{pr_id}: {e}")

reviewers = count_contrib.sorted().reviewer
for login in tqdm(reviewers):
    user_cache[login]

In [ ]:

prev = release_md.merge_bases[0]
other = release_md.merge_bases[1:]
if not other:
    change_str = f"Changes since {prev} follow."
else:
    assert len(other) == 1
    change_str = f"Changes since {prev}, excluding those released in {other[0]}, follow."


notes = release_notes.MarkdownNotes(release_md, note_body)
notes.paragraph(change_str)
notes.sorted_pulls(sorted_pulls)
notes.reviewers(reviewers, user_cache)
notes.changelog_line("celeritas-project", "celeritas")

In [ ]:
import sys
notes.write(sys.stdout)

# Draft github release

In [ ]:
ghapi = cached.api

In [ ]:
release = release_notes.find_release(release_md.version)
if not release:
    release_md = release_notes.create_release(release_md, notes)

In [ ]:
artifact_url, artifact_tgz = release_notes.get_or_upload_tarball(release)
print(f"Artifact URL: {artifact_url}")

In [ ]:
#assert 0
sha256_hash = hashlib.sha256(artifact_tgz).hexdigest()
pyperclip.copy(f'version("{release_md.release}", sha256="{sha256_hash}")')
print("Spack version copied to clipboard!")
subprocess.check_call(
    [
        "open",
        Path(environ["SPACK_ROOT"])
        / "var/spack/repos/builtin/packages/celeritas/package.py",
    ]
)

## TODO: push to zenodo